# Decision Trees
Decision trees are a fairly easy to understand machine learning model that works by making use of thresholds on the independent variables to achieve classification (binary or multiclass).

E.g:
two independent variables: x1 and x2

three classes for y: 0, 1 and 2

Our Decision Tree model could be something as simple as:

```
if x1 < 3:
    y = 2
else:
    if x2 < 5:
        y = 0
    else:
        y = 1
```

The way these values for which the independent variables are compared against, are determined through entropy: the measurement of randomness.

In [11]:
from math import log2

def entropy(values):
    result = 0
    total = sum(values)
    for val in values:
        # ignore 0 values as log(0) is undefined
        if val == 0:
            continue
        # apply entropy formula
        prob = val/total
        result += prob*log2(prob)
    result = max(0.0, -result)
    return result

print (entropy([100, 0])) # No entropy
print (entropy([5, 5])) # full entropy
print (entropy([9, 5])) # high entropy, but there's a tendency
print (entropy([5, 5, 5])) # full entropy for 3 classes

#if we want to normalize entropy:
def normalized_entropy(values):
    result = 0
    total = sum(values)
    for val in values:
        # ignore 0 values as log(0) is undefined
        if val == 0:
            continue
        # apply entropy formula
        prob = val/total
        result += prob*log2(prob)
    # normalize between 0 and 1
    result /= log2(1/len(values))
    return result

print (normalized_entropy([5, 5, 5]))

0.0
1.0
0.9402859586706311
1.584962500721156
0.9999999999999999


### Entropy
Entropy is relevant because decision trees use it as criteria for leaf placement. If a given node in the tree has high entropy, than the tree will branch. When a node has no entropy, the node will become a leaf.

Intuitively, while the distribution of classification in a given is random, we keep branching, until it becomes "deterministic", i.e., non-random.

### Information Gain
Another important concept, which makes use of entropy is Information Gain (IG), which indicates how much a given variable provides information regarding another.

IG(Y,X) = E(Y) - E(Y|X)

The above expression is quite intuitive: "How much entropy did we loose on Y, by acquiring information regarding X", or, dumbing it down further: "Now that I know X, how less random is Y?"

If the entropy value is normalized, IG will always be normalized as well. Regardless of normalization, bigger IG is always a good thing. Unitary IG means that Y is completely random by itself, but completely certain if we know X. Thus, IG can be used to evaluate how much a particular variable is informative of our target classification.

In [12]:
"""
Example: whether to play tennis or not, based on categorical wheather data
"""

data_dict = {
    "Outlook" : [
        "Sunny", "Sunny", "Overcast", "Rain", "Rain", "Rain", "Overcast", "Sunny", "Sunny", "Rain", "Sunny", "Overcast", "Overcast", "Rain"],
    "Temperature" : [
        "Hot", "Hot", "Hot", "Mild", "Cool", "Cool", "Cool", "Mild", "Cool", "Mild", "Mild", "Mild", "Hot", "Mild"],
    "Humidity": [
        "High", "High", "High", "High", "Normal", "Normal", "Normal", "High", "Normal", "Normal", "Normal", "High", "Normal", "High"],
    "Wind": [
        "Weak", "Strong", "Weak", "Weak", "Weak", "Strong", "Strong", "Weak", "Weak", "Weak", "Strong", "Strong", "Weak", "Strong"],
    "PlayTennis": [
        "No", "No", "Yes", "Yes", "Yes", "No", "Yes", "No", "Yes", "Yes", "Yes", "Yes", "Yes", "No"]
}

In [13]:
from collections import Counter
for key in data_dict:
    print(entropy(Counter(data_dict[key]).values()))

1.5774062828523454
1.5566567074628228
1.0
0.9852281360342515
0.9402859586706311


In [14]:
# Let's see, for two variables, the Information Gain we get for PlayTennis

# Outlook
label_counts = Counter(data_dict["PlayTennis"]).values()
raw_entropy = entropy(label_counts)
print (f"\n{raw_entropy = }\n")

#for ease of use, let's convert the dictionary into a pandas DataFrame
import pandas as pd
df = pd.DataFrame(data_dict)

# And now, create a function to calculate IG
def information_gain(independent_variable : str, label_variable : str = "PlayTennis"):
    original_entropy = entropy(label_counts)
    relative_entropy = 0
    total = len(df)
    
    for feature, count in Counter(df[independent_variable]).items():
        df_subset = df.loc[df[independent_variable] == feature]
        label_cntr = Counter(df_subset[label_variable])
        entropy_given_feature = entropy(label_cntr.values())
        relative_entropy += entropy_given_feature*count/total
    return original_entropy-relative_entropy

ig_dict = {
    "Feature" : [],
    "IG" : []
}
for feature in df.drop("PlayTennis", axis=1):
    ig_dict["Feature"].append(feature)
    ig_dict["IG"].append(information_gain(feature))
print (pd.DataFrame(ig_dict).sort_values(by=("IG"), ascending=False))


raw_entropy = 0.9402859586706311

       Feature        IG
0      Outlook  0.246750
2     Humidity  0.151836
3         Wind  0.048127
1  Temperature  0.029223


In [15]:
# Based on the previous result, we can infer that Outlook is the most informative feature, while temperature is near irrelevant. This can be intuitively confirmed, by inspecting the dataframe.

# There should be a relationship between "Yes" and "No", and Outlook feature values:
print (df.sort_values(by="Outlook").drop([i for i in df if i not in ("Outlook", "PlayTennis")], axis=1))
# and indeed, "Outcast" always results in "Yes", while "Rain" and "Sunny" seem to have less impact

# There should NOT be a relationship in the case of Temperature:
print (df.sort_values(by="Temperature").drop([i for i in df if i not in ("Temperature", "PlayTennis")], axis=1))
# and indeed, there is no obvious relationship between any temperature value and "Yes"/"No"

     Outlook PlayTennis
2   Overcast        Yes
6   Overcast        Yes
11  Overcast        Yes
12  Overcast        Yes
3       Rain        Yes
4       Rain        Yes
5       Rain         No
9       Rain        Yes
13      Rain         No
0      Sunny         No
1      Sunny         No
7      Sunny         No
8      Sunny        Yes
10     Sunny        Yes
   Temperature PlayTennis
4         Cool        Yes
5         Cool         No
6         Cool        Yes
8         Cool        Yes
0          Hot         No
1          Hot         No
2          Hot        Yes
12         Hot        Yes
3         Mild        Yes
7         Mild         No
9         Mild        Yes
10        Mild        Yes
11        Mild        Yes
13        Mild         No


## Gini Impurity
Alternatively to IG, Gini Impurity can be used to acess branching, and is useful due to its conceptual simplicity, and resulting computational speed. It is given as:
Igi(Y) = 1 - sum(P(yi)^2)

E.g.

For a given decision tree, a node has the condition x1 > a in its root node. The possible labels are "Yes" and "No".

In the root node itself (before the condition is applied) the number of "Yes" and "No" is the same, and thus, Igi(Y) = 1-(P("Yes")^2 + P("No")^2) = 0.5

Now, the condition is applied, and we move on to the next node (whatever it may be), and in this new node, the number of "Yes" decrease substantially such that its probability is now 0.3. So, now, Igi(Y) = 0.42.

In any given Leaf Node, where there are only "Yes" or "No", we get Igi(Y) = 0 -> we have zero impurity, i.e., the node is "pure".

## Non-categorical data
### Continuous data

What if we want to evaluate a continous variable's information gain? Well, we can't, not directly. Instead intervals in the variable can be defined and considered categories.

E.g.
x1 ∈ [0, 3[

x1 can be made categorical through:

xc1 = 0, if x1 < 1,

xc1 = 1, if 1 <= x1 < 2,

xc1 = 2, if 2 <= x1 < 3

And thus, xc1 is now a categorical variable with 3 possible values, and the information gain from those values can be calculated.

### Integers
Although not usual (nor recommended), integers can be used as categorical variables as well. The reason it's not recommended is due to how the label might be under represented: suppose we have x1 being an integer variable over a huge range, such as [1, 1000], with only two possible resulting labels. In the worst case scenario, if all our data points have a unique integer (no number is repeated) than each integer will be directly associated with its corresponding label, leading the entropy to be always 0, and the information gain to be maximal, even though it's likely that our model will perform poorly on future values based on this feature alone. Our model would effectively be storing a one-to-one table of integer:label pairs. However, for the same scenario, if we are examing over 1 million points, then even this naive approach might yield decent results, because it's impossible for this "one-to-one" correspondance to happen.


## Decison Trees for Regression
Decision trees can also be used for regression, in much the same way as described before for adapting the independent variables. Often, the predicted value is the average of the subset. This means that the resulting prediction is constant within the same branch, resulting in snappy and jittery regression:

![Regression Decision Tree](DTreg.png)
